In [1]:
import pandas as pd, numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve, classification_report
import matplotlib.pyplot as plt

DATA = Path("../data/banksim.csv")
df = pd.read_csv(DATA)

drop_ids = [c for c in df.columns if c.lower().startswith('name')]
X = df.drop(columns=['isFraud'] + drop_ids)
y = df['isFraud'].astype(int)

num_cols = X.select_dtypes(include=['int64','float64']).columns.tolist()
cat_cols = X.select_dtypes(include=['object']).columns.tolist()

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, stratify=y, random_state=42
)

use_iterative = False  # set True to mimic MICE
num_imputer = IterativeImputer(random_state=42) if use_iterative else SimpleImputer(strategy="median")
cat_imputer = SimpleImputer(strategy="most_frequent")

pre = ColumnTransformer(
    transformers=[
        ("num", Pipeline([("imputer", num_imputer)]), num_cols),
        ("cat", Pipeline([("imputer", cat_imputer), ("onehot", OneHotEncoder(handle_unknown="ignore"))]), cat_cols),
    ]
)

clf = Pipeline(steps=[
    ("pre", pre),
    ("logreg", LogisticRegression(max_iter=1000, class_weight="balanced"))
])

clf.fit(X_train, y_train)

probs = clf.predict_proba(X_test)[:,1]
print("ROC AUC:", roc_auc_score(y_test, probs))

fpr, tpr, thr = roc_curve(y_test, probs)
plt.plot(fpr, tpr); plt.plot([0,1],[0,1],'--')
plt.title("ROC Curve"); plt.xlabel("FPR"); plt.ylabel("TPR"); plt.show()

print(classification_report(y_test, (probs>=0.5).astype(int)))

ImportError: IterativeImputer is experimental and the API might change without any deprecation cycle. To use it, you need to explicitly import enable_iterative_imputer:
from sklearn.experimental import enable_iterative_imputer